# Feathr Feature Store on Azure Demo Notebook

This notebook illustrates the use of Feature Store to create a model that predicts NYC Taxi fares. It includes these steps:


This tutorial demonstrates the basic usage of Feathr Feature Store on Azure. It includes these steps:

1. Setup Feathr Environment
2. Initialize Feathr Client
3. Defining Features with Feathr
4. Registering Features to the Feathr feature registry
5. Creating training data point-in-time correct features joins
6. Materializing features to an offline/online store

In this tutorial, we use Feathr Feature Store to create a model that predicts NYC Taxi fares. The dataset comes from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). The feature flow is as below:

![Feature Flow](https://github.com/linkedin/feathr/blob/main/docs/images/feature_flow.png?raw=true)

# Setup Feathr Environment

**REQUIRED STEP: Fill in the resource prefix when provisioning the resources**

***Prior to running the notebook, make sure you have deployed all required resources in your Azure subscription.*** 

In [1]:
# replace with your prefix
resource_prefix = "rifeathr"
storage_accountname = "rifeathrdls"
storage_containername = "nyctaxi"

In [2]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
import pandavro as pdx
from feathr import FeathrClient
from feathr import BOOLEAN, FLOAT, INT32, ValueType
from feathr import Feature, DerivedFeature, FeatureAnchor
from feathr import BackfillTime, MaterializationSettings
from feathr import FeatureQuery, ObservationSettings
from feathr import RedisSink, HdfsSink, JdbcSink
from feathr import INPUT_CONTEXT, HdfsSource
from feathr import WindowAggTransformation
from feathr import TypedKey
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

In [3]:
# Get all the required credentials from Azure Key Vault
key_vault_name=resource_prefix+"kv"
synapse_workspace_url=resource_prefix+"syws"
adls_account=resource_prefix+"dls"
adls_fs_name=resource_prefix+"fs"
purview_name=resource_prefix+"purview"
key_vault_uri = f"https://{key_vault_name}.vault.azure.net"
credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
client = SecretClient(vault_url=key_vault_uri, credential=credential)
secretName = "FEATHR-ONLINE-STORE-CONN"
retrieved_secret = client.get_secret(secretName).value

# Set the resource link
os.environ['spark_config__azure_synapse__dev_url'] = f'https://{synapse_workspace_url}.dev.azuresynapse.net'
os.environ['spark_config__azure_synapse__pool_name'] = 'spark31'
os.environ['spark_config__azure_synapse__workspace_dir'] = f'abfss://{adls_fs_name}@{adls_account}.dfs.core.windows.net/feathr_project'
# os.environ['feature_registry__purview__purview_name'] = f'{purview_name}'
os.environ['FEATURE_REGISTRY__API_ENDPOINT']= f'https://{resource_prefix}webapp.azurewebsites.net/api/v1'
os.environ['online_store__SQL_RIFEATHRDB__USER'] = "sqladmin"
os.environ['online_store__SQL_RIFEATHRDB__PASSWORD'] = "ThisIsNotVerySecure!"
feathr_output_path = f'abfss://{adls_fs_name}@{adls_account}.dfs.core.windows.net/feathr_output'

# Initialize Feathr Client

In [4]:
config_file_name = "feathr_config.yaml" 
feathr_client = FeathrClient(config_path=config_file_name, credential=credential)

2022-12-03 07:21:30.345 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - secrets__azure_key_vault__name not found in the config file.
2022-12-03 07:21:30.403 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - online_store__redis__host not found in the config file.
2022-12-03 07:21:30.440 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - online_store__redis__port not found in the config file.
2022-12-03 07:21:30.467 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - online_store__redis__ssl_enabled not found in the config file.
2022-12-03 07:21:30.675 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - spark_config__azure_synapse__feathr_runtime_location not found in the config file.
2022-12-03 07:21:34.641 | INFO     | feathr.utils._envvariableutil:get_environment_variable:82 - REDIS_PASSWORD is not set in the e

# Defining Features with Feathr

In Feathr, a feature is viewed as a function, mapping from entity id or key, and timestamp to a feature value. For more details on feature definition, please refer to the [Feathr Feature Definition Guide](https://github.com/linkedin/feathr/blob/main/docs/concepts/feature-definition.md)


1. The typed key (a.k.a. entity id) identifies the subject of feature, e.g. a user id, 123.
2. The feature name is the aspect of the entity that the feature is indicating, e.g. the age of the user.
3. The feature value is the actual value of that aspect at a particular time, e.g. the value is 30 at year 2022.


Note that, in some cases, such as features defined on top of request data, may have no entity key or timestamp.
It is merely a function/transformation executing against request data at runtime.
For example, the day of week of the request, which is calculated by converting the request UNIX timestamp.


## Define Sources Section with UDFs
A feature source is needed for anchored features that describes the raw data in which the feature values are computed from. See the python documentation to get the details on each input column.


In [5]:
# define a pre-processing function/UDF to be used on spark 
from pyspark.sql import SparkSession, DataFrame
def feathr_udf_day_calc(df: DataFrame) -> DataFrame:
    from pyspark.sql.functions import dayofweek, dayofyear, col
    df = df.withColumn("fare_amount_cents", col("fare_amount")*100)
    return df

# define the data source
batch_source = HdfsSource(name="nycTaxiBatchSource",
                          path="abfss://{container_name}@{storage_account}.dfs.core.windows.net/nyc_taxi.parquet".format(storage_account=storage_accountname, container_name=storage_containername),
                          event_timestamp_column="lpep_dropoff_datetime",
                          preprocessing=feathr_udf_day_calc,
                          timestamp_format="yyyy-MM-dd HH:mm:ss")

### Define Anchors and Features
A feature is called an anchored feature when the feature is directly extracted from the source data, rather than computed on top of other features. The latter case is called derived feature.

In [6]:
# define anchor features
f_trip_distance = Feature(name="f_trip_distance",
                          feature_type=FLOAT, transform="trip_distance")
f_trip_time_duration = Feature(name="f_trip_time_duration",
                               feature_type=INT32,
                               transform="(to_unix_timestamp(lpep_dropoff_datetime) - to_unix_timestamp(lpep_pickup_datetime))/60")

features = [
    f_trip_distance,
    f_trip_time_duration,
    Feature(name="f_is_long_trip_distance",
            feature_type=BOOLEAN,
            transform="cast_float(trip_distance)>30"),
    Feature(name="f_day_of_week",
            feature_type=INT32,
            transform="dayofweek(lpep_dropoff_datetime)"),
]

request_anchor = FeatureAnchor(name="request_features",
                               source=INPUT_CONTEXT,
                               features=features)

### Window aggregation features

For window aggregation features, see the supported fields below:

Note that the `agg_func` should be any of these:

| Aggregation Type | Input Type | Description |
| --- | --- | --- |
|SUM, COUNT, MAX, MIN, AVG	|Numeric|Applies the the numerical operation on the numeric inputs. |
|MAX_POOLING, MIN_POOLING, AVG_POOLING	| Numeric Vector | Applies the max/min/avg operation on a per entry bassis for a given a collection of numbers.|
|LATEST| Any |Returns the latest not-null values from within the defined time window |


After you have defined features and sources, bring them together to build an anchor:


Note that if the data source is from the observation data, the `source` section should be `INPUT_CONTEXT` to indicate the source of those defined anchors.

In [7]:
location_id = TypedKey(key_column="DOLocationID",
                       key_column_type=ValueType.INT32,
                       description="location id in NYC",
                       full_name="nyc_taxi.location_id")

# calculate the average trip fare, maximum fare and total fare per location for 90 days
agg_features = [Feature(name="f_location_avg_fare",
                        key=location_id,
                        feature_type=FLOAT,
                        transform=WindowAggTransformation(agg_expr="cast_float(fare_amount)",
                                                          agg_func="AVG",
                                                          window="90d")),
                Feature(name="f_location_max_fare",
                        key=location_id,
                        feature_type=FLOAT,
                        transform=WindowAggTransformation(agg_expr="cast_float(fare_amount)",
                                                          agg_func="MAX",
                                                          window="90d")),
                Feature(name="f_location_total_fare_cents",
                        key=location_id,
                        feature_type=FLOAT,
                        transform=WindowAggTransformation(agg_expr="fare_amount_cents",
                                                          agg_func="SUM",
                                                          window="90d")),
                ]

agg_anchor = FeatureAnchor(name="aggregationFeatures",
                           source=batch_source,
                           features=agg_features)

### Derived Features 
Derived features are the features that are computed from other features. They could be computed from anchored features, or other derived features.

In [8]:
f_trip_time_distance = DerivedFeature(name="f_trip_time_distance",
                                      feature_type=FLOAT,
                                      input_features=[
                                          f_trip_distance, f_trip_time_duration],
                                      transform="f_trip_distance * f_trip_time_duration")

f_trip_time_rounded = DerivedFeature(name="f_trip_time_rounded",
                                     feature_type=INT32,
                                     input_features=[f_trip_time_duration],
                                     transform="f_trip_time_duration % 10")


And then we need to build those features so that it can be consumed later. Note that we have to build both the "anchor" and the "derived" features (which is not anchored to a source).

In [9]:
feathr_client.build_features(anchor_list=[agg_anchor, request_anchor], derived_feature_list=[
                      f_trip_time_distance, f_trip_time_rounded])

### Registering features

We can also register the features with an Apache Atlas compatible service, such as Azure Purview, and share the registered features across teams:

In [18]:
credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)


In [13]:
feathr_client.register_features()
# feathr_client._create_project()

RuntimeError: Failed to call registry API, status is 400, error is <html>
<head><title>400 Request Header Or Cookie Too Large</title></head>
<body>
<center><h1>400 Bad Request</h1></center>
<center>Request Header Or Cookie Too Large</center>
<hr><center>nginx/1.18.0</center>
</body>
</html>


We can now list the registered features:

In [11]:
feathr_client.list_registered_features(project_name="nyctaxifs")

[{'name': 'f_trip_time_rounded',
  'id': '544cdf6d-aaf4-4fe6-b013-f8939c0c73a5',
  'qualifiedName': 'nyctaxifs__f_trip_time_rounded'},
 {'name': 'f_trip_time_distance',
  'id': '298da136-0db4-49d8-8807-689b4b1076d9',
  'qualifiedName': 'nyctaxifs__f_trip_time_distance'},
 {'name': 'f_trip_time_duration',
  'id': '81864281-ee90-4361-b618-f66bd5d4d32b',
  'qualifiedName': 'nyctaxifs__request_features__f_trip_time_duration'},
 {'name': 'f_is_long_trip_distance',
  'id': '4e69f9fc-bb6b-4c0f-8499-212b226e7062',
  'qualifiedName': 'nyctaxifs__request_features__f_is_long_trip_distance'},
 {'name': 'f_day_of_week',
  'id': '7658ed88-491b-4670-b8a2-3e608d0c4652',
  'qualifiedName': 'nyctaxifs__request_features__f_day_of_week'},
 {'name': 'f_trip_distance',
  'id': '6da0dca1-30de-4523-8c7e-a862df4120c8',
  'qualifiedName': 'nyctaxifs__request_features__f_trip_distance'},
 {'name': 'f_location_max_fare',
  'id': '54eb9548-0a89-4297-b75c-f64f161ecae0',
  'qualifiedName': 'nyctaxifs__aggregationFea

## Create training data using point-in-time correct feature join (e.g usage path: ML model training and scoring)

A training dataset usually contains entity id columns, multiple feature columns, event timestamp column and label/target column. 

To create a training dataset using Feathr, one needs to provide a feature join configuration file to specify
what features and how these features should be joined to the observation data. 

To learn more on this topic, please refer to [Point-in-time Correctness](https://github.com/linkedin/feathr/blob/main/docs/concepts/point-in-time-join.md)


In [13]:

output_path = feathr_output_path

# retrieve a list of features with help of the common key
feature_query = FeatureQuery(
    feature_list=["f_location_avg_fare", "f_trip_time_rounded", "f_is_long_trip_distance", "f_location_total_fare_cents"], key=location_id)

# Time settings of the observation data. Used in feature join (so the data useed for join is this data in observeration path combined with feature list described above)
settings = ObservationSettings(
    observation_path="abfss://{container_name}@{storage_account}.dfs.core.windows.net/nyc_taxi.parquet".format(storage_account=storage_accountname, container_name=storage_containername),
    event_timestamp_column="lpep_dropoff_datetime",
    timestamp_format="yyyy-MM-dd HH:mm:ss")
feathr_client.get_offline_features(observation_settings=settings,
                            feature_query=feature_query,
                            output_path=output_path)
feathr_client.wait_job_to_finish(timeout_sec=500)

2022-12-01 17:13:15.373 | INFO     | feathr.spark_provider._synapse_submission:upload_or_get_cloud_path:67 - Uploading C:\Users\RISENG~1\AppData\Local\Temp\tmpq8_162q6\feathr_pyspark_driver.py to cloud..
2022-12-01 17:13:15.373 | INFO     | feathr.spark_provider._synapse_submission:upload_file:412 - Uploading file feathr_pyspark_driver.py
2022-12-01 17:13:52.704 | INFO     | feathr.spark_provider._synapse_submission:upload_file:418 - C:\Users\RISENG~1\AppData\Local\Temp\tmpq8_162q6\feathr_pyspark_driver.py is uploaded to location: abfss://rifeathrfs@rifeathrdls.dfs.core.windows.net/feathr_project/feathr_pyspark_driver.py
2022-12-01 17:13:52.704 | INFO     | feathr.spark_provider._synapse_submission:upload_or_get_cloud_path:71 - C:\Users\RISENG~1\AppData\Local\Temp\tmpq8_162q6\feathr_pyspark_driver.py is uploaded to location: abfss://rifeathrfs@rifeathrdls.dfs.core.windows.net/feathr_project/feathr_pyspark_driver.py
2022-12-01 17:13:52.736 | INFO     | feathr.spark_provider._synapse_sub

Download the result and show the result

Let's use the helper function `get_result_df` to download the result and view it:

In [14]:
def get_result_df(client: FeathrClient) -> pd.DataFrame:
    """Download the job result dataset from cloud as a Pandas dataframe."""
    res_url = feathr_client.get_job_result_uri(block=True, timeout_sec=600)
    tmp_dir = tempfile.TemporaryDirectory()
    feathr_client.feathr_spark_launcher.download_result(result_path=res_url, local_folder=tmp_dir.name)
    dataframe_list = []
    # assuming the result are in avro format
    for file in glob.glob(os.path.join(tmp_dir.name, '*.avro')):
        dataframe_list.append(pdx.read_avro(file))
    vertical_concat_df = pd.concat(dataframe_list, axis=0)
    tmp_dir.cleanup()
    return vertical_concat_df

df_res = get_result_df(client)
df_res

2022-12-01 17:18:15.423 | INFO     | feathr.spark_provider._synapse_submission:wait_for_completion:177 - Current Spark job status: success
2022-12-01 17:18:15.553 | INFO     | feathr.spark_provider._synapse_submission:download_file:430 - Beginning reading of results from abfss://rifeathrfs@rifeathrdls.dfs.core.windows.net/feathr_output
2022-12-01 17:20:58.594 | INFO     | feathr.spark_provider._synapse_submission:download_file:459 - Finish downloading files from abfss://rifeathrfs@rifeathrdls.dfs.core.windows.net/feathr_output to C:\Users\RISENG~1\AppData\Local\Temp\tmpg9b7cyye.


,trip_id,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,f_is_long_trip_distance,f_location_total_fare_cents,f_location_avg_fare,f_trip_time_rounded
0,25281,NaN,2020-04-03 07:58:00,2020-04-03 08:26:00,None,NaN,47,125,NaN,12.98,...,None,0.3,32.00,NaN,NaN,NaN,False,3170.0,31.700001,8
1,26724,NaN,2020-04-07 22:45:00,2020-04-07 23:08:00,None,NaN,228,125,NaN,7.13,...,None,0.3,31.58,NaN,NaN,NaN,False,6023.0,30.115002,3
2,6453,2.0,2020-04-08 04:09:53,2020-04-08 04:27:48,N,1.0,75,125,1.0,5.93,...,None,0.3,25.05,1.0,1.0,2.75,False,7923.0,26.410002,7
3,8885,1.0,2020-04-10 21:33:57,2020-04-10 22:00:11,N,1.0,89,125,1.0,0.00,...,None,0.3,29.90,1.0,1.0,2.5,False,12363.0,24.726002,6
4,27764,NaN,2020-04-10 21:33:57,2020-04-10 22:00:11,None,NaN,89,125,NaN,0.00,...,None,0.3,31.90,NaN,NaN,NaN,False,12363.0,24.726002,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,35356,NaN,2020-04-30 16:12:00,2020-04-30 17:16:00,None,NaN,55,119,NaN,25.74,...,None,0.3,68.85,NaN,NaN,NaN,False,337156.0,15.325272,4
220,35424,NaN,2020-04-30 17:05:00,2020-04-30 17:25:00,None,NaN,242,119,NaN,6.89,...,None,0.3,21.27,NaN,NaN,NaN,False,338978.0,15.338370,0
221,24371,2.0,2020-04-30 18:09:34,2020-04-30 18:20:12,N,5.0,74,119,1.0,2.98,...,None,0.3,13.30,2.0,2.0,0.0,False,340278.0,15.327837,0
222,35458,NaN,2020-04-30 18:13:00,2020-04-30 18:25:00,None,NaN,127,119,NaN,2.39,...,None,0.3,13.54,NaN,NaN,NaN,False,341327.0,15.306143,2


## Materialize feature value into offline/online storage (e.g usage during: ML model inference path)

While Feathr can compute the feature value from the feature definition on-the-fly at request time, it can also pre-compute
and materialize the feature value to offline and/or online storage. 

We can push the generated features to the online store like below:


In [14]:
from feathr import JdbcSink

backfill_time = BackfillTime(start=datetime(2020, 5, 20), end=datetime(2020, 5, 20), step=timedelta(days=1))

redisSink = RedisSink(table_name="nyctaxi")

name = 'nyctaxifeatures'
SQLSink = JdbcSink(name, "jdbc:sqlserver://rifeathrdbserver.database.windows.net:1433;database=rifeathrdb;", "dbo.nyctaxifs", "USERPASS")

# os.environ[f"{name.upper()}_USER"] = "sqladmin"
# os.environ[f"{name.upper()}_PASSWORD"] = "ThisIsNotVerySecure!"

settings = MaterializationSettings(name="nycTaxiFeatures",
                                   backfill_time=backfill_time,
                                   sinks=[SQLSink],
                                   feature_names=["f_location_avg_fare", "f_location_max_fare"])

feathr_client.materialize_features(settings)
feathr_client.wait_job_to_finish(timeout_sec=500)


2022-12-02 07:53:45.949 | INFO     | feathr.spark_provider._synapse_submission:upload_or_get_cloud_path:67 - Uploading C:\Users\RISENG~1\AppData\Local\Temp\tmp4sb7txp9\feathr_pyspark_driver.py to cloud..
2022-12-02 07:53:45.951 | INFO     | feathr.spark_provider._synapse_submission:upload_file:412 - Uploading file feathr_pyspark_driver.py
2022-12-02 07:54:25.574 | INFO     | feathr.spark_provider._synapse_submission:upload_file:418 - C:\Users\RISENG~1\AppData\Local\Temp\tmp4sb7txp9\feathr_pyspark_driver.py is uploaded to location: abfss://rifeathrfs@rifeathrdls.dfs.core.windows.net/feathr_project/feathr_pyspark_driver.py
2022-12-02 07:54:25.574 | INFO     | feathr.spark_provider._synapse_submission:upload_or_get_cloud_path:71 - C:\Users\RISENG~1\AppData\Local\Temp\tmp4sb7txp9\feathr_pyspark_driver.py is uploaded to location: abfss://rifeathrfs@rifeathrdls.dfs.core.windows.net/feathr_project/feathr_pyspark_driver.py
2022-12-02 07:54:25.574 | INFO     | feathr.utils._envvariableutil:get_

RuntimeError: Spark job failed.

We can then get the features from the online store (Redis in our case):


In [15]:
res = feathr_client.multi_get_online_features("nyctaxifs", ["128", "243"], ["f_location_avg_fare", "f_location_max_fare"])
res

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [13]:
feathr_client.get_online_features("dbo.nyctaxifs", "128", ["f_location_avg_fare", "f_location_max_fare"])

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [16]:
import os

print(os.environ)

environ({'ALLUSERSPROFILE': 'C:\\ProgramData', 'APPDATA': 'C:\\Users\\risengupta\\AppData\\Roaming', 'APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL': 'true', 'CHOCOLATEYINSTALL': 'C:\\ProgramData\\chocolatey', 'CHOCOLATEYLASTPATHUPDATE': '133131757029497056', 'CHOCOLATEYTOOLSLOCATION': 'C:\\tools', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files', 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files', 'COMPUTERNAME': 'RJSURFACEBOOK', 'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe', 'CONDA_DEFAULT_ENV': 'datascience-env', 'CONDA_EXE': 'C:\\Users\\risengupta\\Miniconda3\\Scripts\\conda.exe', 'CONDA_PREFIX': 'C:\\Users\\risengupta\\Miniconda3\\envs\\datascience-env', 'CONDA_PREFIX_1': 'C:\\Users\\risengupta\\Miniconda3', 'CONDA_PROMPT_MODIFIER': '(datascience-env) ', 'CONDA_PYTHON_EXE': 'C:\\Users\\risengupta\\Miniconda3\\python.exe', 'CONDA_SHLVL': '2', 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData', 'ELECT